In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Scikit-learn for preprocessing and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# TensorFlow / Keras for neural networks
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


# Load data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [ ]:
print(daily['dteday'].max())
print(daily.tail())


In [ ]:
#Chunk 1: Data Preparation (Updated for Dictionary)

# Parse date and create datetime features
bikes['dteday'] = pd.to_datetime(bikes['dteday'])

# Group by day to get total rentals per day
daily = bikes.groupby('dteday').agg({
    'temp_c': 'mean',
    'feels_like_c': 'mean',
    'hum': 'mean',
    'windspeed': 'mean',
    'holiday': 'first',
    'workingday': 'first',
    'season': 'first',
    'weathersit': 'mean',
    'casual': 'sum',
    'registered': 'sum'
}).reset_index()

# Create total count
daily['count'] = daily['casual'] + daily['registered']

# Time-based features
daily['day_of_week'] = daily['dteday'].dt.dayofweek
daily['month'] = daily['dteday'].dt.month

# Cyclical encoding for day of week and month
daily['day_sin'] = np.sin(2 * np.pi * daily['day_of_week'] / 7)
daily['day_cos'] = np.cos(2 * np.pi * daily['day_of_week'] / 7)
daily['month_sin'] = np.sin(2 * np.pi * daily['month'] / 12)
daily['month_cos'] = np.cos(2 * np.pi * daily['month'] / 12)

# Final features
features = ['temp_c', 'feels_like_c', 'hum', 'windspeed', 'holiday',
            'workingday', 'season', 'weathersit', 'day_sin', 'day_cos',
            'month_sin', 'month_cos']
target = 'count'

# Use October for testing
train_bikes = daily[daily['dteday'] < '2023-10-01']
test_bikes = daily[(daily['dteday'] >= '2023-10-01') & (daily['dteday'] <= '2023-10-31')]


X_train = train_bikes[features]
y_train = train_bikes[target]
X_test = test_bikes[features]
y_test = test_bikes[target]

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
#Chunk 2: Model Training, Prediction, and Visualization

# Build the model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train,
                    validation_split=0.2,
                    epochs=100,
                    callbacks=[early_stop],
                    verbose=1)

# Predict on test data (Nov–Dec)
predictions = model.predict(X_test_scaled).flatten()

# Create results DataFrame
results = test_bikes[['dteday']].copy()
results['predicted_count'] = predictions
results['actual_count'] = y_test.values

# Save predictions to CSV
results.to_csv('bike_predictions_nov_dec.csv', index=False)

# Plot predictions vs actuals
plt.figure(figsize=(10,5))
plt.plot(results['dteday'], results['actual_count'], label='Actual')
plt.plot(results['dteday'], results['predicted_count'], label='Predicted')
plt.xlabel('Date')
plt.ylabel('Bike Rentals')
plt.title('Predicted vs Actual Rentals (Nov–Dec)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

